Этот скрипт предназначен для анализа составленной базы данных: интересуют характеристики активности рынков: как в целом, так и в срезах по тикерам.

In [56]:
from psycopg2 import connect as cnct
from datetime import datetime

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os

# 0. Подготовка глобальных переменных

Здесь записывается параметры подключения к базе данных, ее имя, а также рассчитываются основные глобальные параметры:
1. Названия всех таблиц в базе данных
2. Все тикеры инструментов, присутствующие в данных

In [2]:
DB_NAME = 'orderlogs'
DB_SETTINGS = {
    'dbname': DB_NAME,
    'user': 'postgres',
    'password': '',
    'host': 'localhost',
    'port': 5432
}

with cnct(**DB_SETTINGS) as conn:
    cur = conn.cursor()
    cur.execute("SELECT table_name FROM information_schema.tables WHERE table_schema='public'")
    TABLE_NAMES = cur.fetchall()
    TABLE_NAMES = sorted(list(map(lambda x: x[0], TABLE_NAMES)))

In [3]:
SECCODES = []
for table_name in TABLE_NAMES:
    with cnct(**DB_SETTINGS) as conn:
        cur = conn.cursor()
        cur.execute(f'SELECT DISTINCT "SECCODE" FROM "{table_name}"')
        SECCODES.extend(list(map(lambda x: x[0], cur.fetchall())))
SECCODES = sorted(list(np.unique(np.array(SECCODES))))

In [4]:
def date_convertion(date):
    '''This function converts date in int format like 20190101 to POSIX
    
    :date: date in int format
    '''
    year = int(str(date)[:4])
    month = int(str(date)[4:6])
    day = int(str(date)[6:])
    
    return datetime(year, month, day, )

# 1. Оценка размера ежедневной активности

## 1.1. Общая активность

Произведем расчет количества заявок по рынку в целом, чтобы было представление об активности на рынках. Здесь помимо фундаментально торгующих инвесторов так же присутствует информация о HFT торгах: активность алгоритмической торговли. 

In [ ]:
activity_value = []
for table_name in TABLE_NAMES:
    print(table_name)
    with cnct(**DB_SETTINGS) as conn:
        cur = conn.cursor()
        cur.execute(f'SELECT COUNT(*) from "{table_name}"')
        orders_count = cur.fetchone()[0]  # the return is tuple, we need the first value
        activity_value.append((table_name, orders_count))
activity_value = pd.DataFrame(activity_value, columns=['Date', 'Number of Orders'])
activity_value['Date'] = list(map(date_convertion, activity_value['Date']))

In [ ]:
activity_value.to_csv('./data_results/description/tables_activity/activity_market_all.csv', index=False)

In [ ]:
plt.figure()
plt.plot(activity_value['Date'],
         activity_value['Number of Orders'])
plt.xticks(rotation=15)
plt.title('The Activity of Stock Market, num of orders')

plt.savefig('./data_results/description/figures_activity/activity_stock_market_all.pdf', )

## 1.2. Активность в разрезе по тикерам

Аналогичный расчет подготовим по всем финансовым инструментам, по которым была активность по составленной БД (июнь 2019). 

In [ ]:
def activity_calculation(seccode):
    '''This function calculates the market activity by ticker (by fin. instrument).
    
    :seccode: str parameter (for example, 'SBER')
    :return: pd.DataFrame object with columns Date, Seccode, Number of Orders; this dataframe is saved to the
        .csv file to the further usage
    '''
    activity_value_tickers = []
    for table_name in TABLE_NAMES:
        print(table_name, seccode)
        with cnct(**DB_SETTINGS) as conn:
            cur = conn.cursor()
            try:
                cur.execute(f'''SELECT COUNT(*) from "{table_name}" WHERE "SECCODE" = '{seccode}';''')
                count = cur.fetchone()[0]
                activity_value_tickers.append((table_name, seccode, count))
            except:
                pass
    activity_value_tickers = pd.DataFrame(activity_value_tickers, columns=['Date', 'Seccode', 'Number of Orders'])
    activity_value_tickers['Date'] = list(map(date_convertion, activity_value_tickers['Date']))
    
    activity_value_tickers.to_csv(f'./data_results/description/tables_activity/activity_market_{seccode}.csv', index=False)

    return activity_value_tickers

In [ ]:
def activity_plot(activity_value_ticker, seccode):
    '''This function plots the result of activity_calculation function
    
    :activity_value_ticker: pd.DataFrame object that is return of activity_calculation function
    :seccode: str name of fin. instrument (for example, 'SBER')
    :return: None, the plot is saved into .pdf file 
    '''
    plt.figure()
    plt.plot(activity_value_ticker['Date'],
             activity_value_ticker['Number of Orders'])
    plt.xticks(rotation=15)
    plt.title(f'The Activity of {seccode}, num of orders')

    plt.savefig(f'./data_results/description/figures_activity/activity_stock_market_{seccode}.pdf')
    plt.close

In [ ]:
for seccode in SECCODES:
    activity_data = activity_calculation(seccode)
    activity_plot(activity_data, seccode)

# 2. Анализ объема торгов

Здесь необходима оценка глубины рынков как общая, так и в разрезе по тикерам. Оценка глубины показывает денежное выражение в заключенных сделках. По полученной информации можно отфильтровать большое число инструментов для учета только наиболее ликвидных инструментов как тех, которые вносят больший вклад в качество рынка. 

## 2.1. Общий объем торгов

In [9]:
def sum_volume_market_day(table_name):
    '''This function calculates the overall traded volume in 
    money that occurs during the day that is specified in the table
    
    :table_name: the name of table in the database ('20190603' for example)
    :return: float number that is overall traded volume
    '''
    with cnct(**DB_SETTINGS) as conn:
        cur = conn.cursor()
        query = f'SELECT sum("VOLUME" * "TRADEPRICE") FROM "{table_name}" WHERE ' +\
            f'"{table_name}"."ACTION"=2 ' +\
            f'AND "{table_name}"."BUYSELL"=\'B\';'

        cur.execute(query)

        result = cur.fetchall()[0][0]  # return is the list consists of tuple with 1 value
    return result


def sum_volume_market_month():
    '''This function calculates the same as the sum_volume_market_day function but for all the days that
    exist in the data base. 
    
    :return: the pd.DataFrame object with columns Date, TRADEDVOLUME
    '''
    df_return = []
    for table_name in TABLE_NAMES:
        print(table_name)
        try:
            volume_day = sum_volume_market_day(table_name)
            if np.float(volume_day) == np.nan:
                volume_day = 0
        except:
            volume_day = 0
    
        df_return.append((date_convertion(table_name), volume_day))
    
    return pd.DataFrame(df_return, columns=['Date', 'TRADEDVOLUME'])


def tradedvolume_plot_market(df):
    '''This function plots the result of sum_volume_market_month
    
    :df: pd.DataFrame object that is return of sum_volume_market_month function
    :return: None, the plot is saved into .pdf file 
    '''
    plt.figure()
    plt.plot(df['Date'],
             df['TRADEDVOLUME'])
    plt.xticks(rotation=15)
    plt.title(f'The Traded Volume of Market, rub')

    plt.savefig(f'./data_results/description/figures_tradedvolume/tradedvolume_market.pdf')
    plt.close()

In [11]:
df = sum_volume_market_month()
df.to_csv(f'./data_results/description/tables_tradedvolume/tradedvolume_market.csv', index=False)
tradedvolume_plot_market(df)

20190603
20190604
20190605
20190606
20190607
20190610
20190611
20190613
20190614
20190617
20190618
20190619
20190620
20190621
20190624
20190625
20190626
20190627
20190628


/usr/local/anaconda3/lib/python3.7/site-packages/pandas/plotting/_matplotlib/converter.py:103: FutureWarning: Using an implicitly registered datetime converter for a matplotlib plotting method. The converter was registered by pandas on import. Future versions of pandas will require you to explicitly register matplotlib converters.

To register the converters:
	>>> from pandas.plotting import register_matplotlib_converters
	>>> register_matplotlib_converters()
  warnings.warn(msg, FutureWarning)


## 2.2. Объем торгов в разрезе по тикерам

In [ ]:
def sum_volume_ticker_day(ticker, table_name):
    '''This function calculates the traded volume in money by ticker at the specified date
    
    :ticker: str name of fin. instrument (for example, 'SBER')
    :table_name: the name of postgresql table (for example, '20190603')
    :return: the float number of sum of daily traded volume (in money, rub)
    '''
    with cnct(**DB_SETTINGS) as conn:
        cur = conn.cursor()
        query = f'SELECT sum("VOLUME" * "TRADEPRICE") FROM "{table_name}" WHERE ' +\
            f'"{table_name}"."ACTION"=2 ' +\
            f'AND "{table_name}"."SECCODE"=\'{ticker}\' ' +\
            f'AND "{table_name}"."BUYSELL"=\'B\';'

        cur.execute(query)

        result = cur.fetchall()[0][0]  # return is the list consists of tuple with 1 value
    
    return result


def sum_volume_ticker_month(ticker):
    '''This function calculates the same as the sum_volume_ticker_day function but for all the days that
    exist in the data base. 
    
    :ticker: str name of fin. instrument (for example, 'SBER')
    :return: the pd.DataFrame object with columns Date, SECCODE, TRADEDVOLUME
    '''
    df_return = []
    for table_name in TABLE_NAMES:
        print(ticker, table_name)
        try:
            volume_day = sum_volume_ticker_day(ticker, table_name)
            if np.float(volume_day) == np.nan:
                volume_day = 0
        except:
            volume_day = 0
    
        df_return.append((date_convertion(table_name), ticker, volume_day))
    
    return pd.DataFrame(df_return, columns=['Date', 'SECCODE', 'TRADEDVOLUME'])


def tradedvolume_plot(df, seccode):
    '''This function plots the result of sum_volume_ticker_month
    
    :df: pd.DataFrame object that is return of sum_volume_ticker_month function
    :seccode: str name of fin. instrument (for example, 'SBER')
    :return: None, the plot is saved into .pdf file 
    '''
    plt.figure()
    plt.plot(df['Date'],
             df['TRADEDVOLUME'])
    plt.xticks(rotation=15)
    plt.title(f'The Traded Volume of {seccode}, rub')

    plt.savefig(f'./data_results/description/figures_tradedvolume/tradedvolume_{seccode}.pdf')
    plt.close()

In [ ]:
for seccode in SECCODES:
    df = sum_volume_ticker_month(seccode)
    df.to_csv(f'./data_results/description/tables_tradedvolume/tradedvolume_{seccode}.csv', index=False)
    tradedvolume_plot(df, seccode)

# 3. Отбор инструментов

In [91]:
path = os.path.join(os.path.abspath(''), 'data_results/description/tables_tradedvolume')
filenames = os.listdir(path)
filenames = sorted(list(filter(lambda x: x[-3:]=='csv', filenames)))

df = pd.DataFrame(columns=['Date', 'SECCODE', 'TRADEDVOLUME'])
for file in filenames:
    if file != 'tradedvolume_market.csv':
        df_readed = pd.read_csv(os.path.join(path, file))
        df = pd.concat((df, df_readed))
    else:
        pass
df = df.reset_index(drop=True).sort_values(by=['SECCODE', 'Date'])

df_overall_sorted = df.groupby('SECCODE').agg({
    'Date': lambda x: datetime.strptime(x.iloc[-1][:-3], '%Y-%m'),
    'TRADEDVOLUME': np.sum
}).drop('Date', axis=1).sort_values(by='TRADEDVOLUME', ascending=False).reset_index()
df_overall_sorted['% of total'] = df_overall_sorted['TRADEDVOLUME'] / np.sum(df_overall_sorted.TRADEDVOLUME) * 100

df_overall_sorted['% of total cumulative'] = np.cumsum(df_overall_sorted['% of total'])
df_overall_sorted.to_csv('./data_results/selection/volume_partition.csv')

Оставляем топ 58 тикеров, которые за все время торгов составляют хотя бы 0.5% от всего рынка. Кумулятивно такие тикеры вместе показывают 99.1% рынка. 

In [122]:
seccode_for_spec = sorted(df_overall_sorted['SECCODE'][0:58])

path_to_spec = os.path.join(os.path.abspath(''), 'data_from_moex_web')

spec = pd.read_excel(os.path.join(path_to_spec, 'list-archive-01062019.xlsx'), header=1)
filter_ = np.array(list(map(lambda x: x in seccode_for_spec, spec['Торговый код'])))
spec = spec.loc[filter_].reset_index(drop=True)
spec

,Листинг,Дата включения в список,Эмитент,ИНН,Ценная бумага,Тип ценной бумаги,Гос. рег. номер,Дата регистрации,Номинал,Объем,Торговый код,Дата погашения,Дата обращения,Предназначение для квалинвесторов
0,Первый уровень,2014-06-09,Polymetal International plc (Полиметалл Интерн...,10033331,Акции обыкновенные бездокументарные именные,Акции иностранного эмитента,NaN,NaT,NaN,469 506 028,POLY,NaT,NaT,NaN
1,Первый уровень,2014-06-09,"АК ""АЛРОСА"" (ПАО)",1433000147,акции обыкновенные,Акция обыкновенная,1-03-40046-N,2011-08-25,0.500000,7 364 965 630,ALRS,NaT,NaT,NaN
2,Первый уровень,2014-06-09,Банк ВТБ (ПАО),7702070139,Акции обыкновенные,Акция обыкновенная,10401000B,2006-09-29,0.010000,12 960 541 337 338,VTBR,NaT,NaT,NaN
3,Первый уровень,2014-06-09,"ПАО ""Аэрофлот""",7712040126,Акции обыкновенные,Акция обыкновенная,1-01-00010-A,2004-01-23,1.000000,1 110 616 299,AFLT,NaT,NaT,NaN
4,Первый уровень,2014-06-09,"ПАО ""ГМК ""Норильский никель""",8401005730,акция обыкновенная бездокументарная именная,Акция обыкновенная,1-01-40155-F,2006-12-12,1.000000,158 245 476,GMKN,NaT,NaT,NaN
5,Первый уровень,2014-06-09,"ПАО ""Газпром""",7736050003,Акции обыкновенные,Акция обыкновенная,1-02-00028-A,1998-12-30,5.000000,23 673 512 900,GAZP,NaT,NaT,NaN
6,Первый уровень,2014-06-09,"ПАО ""Группа Компаний ПИК""",7713011336,Акции обыкновенные,Акция обыкновенная,1-02-01556-A,1998-11-10,62.500000,660 497 344,PIKK,NaT,NaT,NaN
7,Первый уровень,2014-06-09,"ПАО ""Группа ЛСР""",7838360491,Акции обыкновенные,Акция обыкновенная,1-01-55234-E,2006-09-28,0.250000,103 030 215,LSRG,NaT,NaT,NaN
8,Первый уровень,2014-06-09,"ПАО ""ЛУКОЙЛ""",7708004767,Акции обыкновенные,Акция обыкновенная,1-01-00077-A,2003-06-25,0.025000,750 000 000,LKOH,NaT,NaT,NaN
9,Первый уровень,2014-06-09,"ПАО ""М.видео""",7707602010,Акции обыкновенные,Акция обыкновенная,1-02-11700-A,2007-08-23,10.000000,179 768 227,MVID,NaT,NaT,NaN


Поскольку паевые фонды и страховые компании (крупные игроки рынка) имеют право работать только с тикерами первого уровня листинга, то из всего списка (58 тикеров) оставляем 45 верхних, принадлежащих первому уровню листинга.

In [127]:
spec = spec.loc[spec['Листинг']=='Первый уровень']

Почти итоговый список выбранных инструментов

In [130]:
spec[['Эмитент', 'Тип ценной бумаги', 'Торговый код']].sort_values(
    by='Торговый код').reset_index(drop=True)

,Эмитент,Тип ценной бумаги,Торговый код
0,"ПАО АФК ""Система""",Акция обыкновенная,AFKS
1,"ПАО ""Аэрофлот""",Акция обыкновенная,AFLT
2,"АК ""АЛРОСА"" (ПАО)",Акция обыкновенная,ALRS
3,"ПАО ""МОСКОВСКИЙ КРЕДИТНЫЙ БАНК""",Акция обыкновенная,CBOM
4,"ПАО ""Северсталь""",Акция обыкновенная,CHMF
5,"ПАО ""Детский мир""",Акция обыкновенная,DSKY
6,"ПАО ""Энел Россия""",Акция обыкновенная,ENRU
7,"ПАО ""ФСК ЕЭС""",Акция обыкновенная,FEES
8,Икс 5 Ритейл Груп Н.В.,Депозитарные расписки иностранного эмитента на...,FIVE
9,"ПАО ""Газпром""",Акция обыкновенная,GAZP


Есть выпадающий актив: пай Сбербанка "Управление Активами". Его стоит исключить, поскольку фактически это портфель из активов. Итоговый список: 44 финансового инструмента. 

In [135]:
spec = spec.loc[spec['Торговый код']!='SBGB'].reset_index(drop=True)

spec.to_excel('./data_results/selection/volume_instruments_specification.xlsx', index=False)